#### Imports

In [1]:
import pandas as pd
import numpy as np

#### Carregando os dados

In [2]:
df_relatorio_vendas = (
    pd.read_csv('./dataset/relatorio_vendas_etapa_3.csv', sep=';')
)

df_calendario_china = (
    pd.read_csv('./dataset/calendario_china.csv', sep=';')
)

#### Criando as novas Features


In [3]:
# Feriados
df_relatorio_vendas = (
    df_relatorio_vendas
        .merge(
            df_calendario_china
            ,how = 'left'
            ,left_on = pd.to_datetime(df_relatorio_vendas['DT_ORDER_DATE'])
            ,right_on = pd.to_datetime(df_calendario_china['DATE'])
        )
)

# Verão do Hemisfério Norte
conditions = [
    (
        # Verão do hemisfério norte 2020
        pd.to_datetime(
            df_relatorio_vendas['DT_ORDER_DATE']
        )
        .isin(
            pd.date_range(
                start='2020-06-01'
                ,end='2020-08-31'
            )
        )
    )
    ,(
        # Verão do hemisfério norte 2021
        pd.to_datetime(
            df_relatorio_vendas['DT_ORDER_DATE']
        )
        .isin(
            pd.date_range(
                start='2021-06-01'
                ,end='2021-08-31'
            )
        )
    )
    ,(
        # Verão do hemisfério norte 2022
        pd.to_datetime(
            df_relatorio_vendas['DT_ORDER_DATE']
        )
        .isin(
            pd.date_range(
                start='2022-06-01'
                ,end='2022-08-31'
            )
        )
    )
]

choices = [1,1,1]

df_relatorio_vendas['IN_IS_SUMMER'] = np.select(conditions, choices, default=0)


#### Organizando o dataframe

In [5]:
# Salvando o nome dos feriados para análise futura
df_relatorio_vendas['TEMP'] = (
    df_relatorio_vendas['DS_HOLIDAY_NAME']
    .copy()
)

# Criando as colunas dummy
df_relatorio_vendas = (
    pd.get_dummies(
        df_relatorio_vendas
        ,columns=['TEMP']
        ,prefix = 'IN_IS'
    )
)

# Mostrando as colunas do dataframe após a modificação
df_relatorio_vendas.columns

# Criando uma nova categoria chamada NON_HOLIDAY
(
    df_relatorio_vendas['DS_HOLIDAY_NAME']
      .fillna('NON_HOLIDAY', inplace = True)
)
# Criando uma nova coluna dummy para facilitar a análise exploratória
df_relatorio_vendas['IN_IS_HOLIDAY'] = (
    np.where(
        df_relatorio_vendas['DS_HOLIDAY_NAME'] == 'NON_HOLIDAY'
        ,0 
        ,1
    )
)

### Exportando o dataframe devidamente organizado para ser utilizado no próximo notebook

In [ ]:
# Removendo colunas desnecessárias
df_relatorio_vendas = (
    df_relatorio_vendas[
        df_relatorio_vendas
            .columns[
                ~df_relatorio_vendas
                    .columns.isin(
                            [
                                'key_0'
                                ,'VL_ORDER_PRICE'
                                ,'VL_AVERAGE_UNIT_RETAIL'
                                ,'DATE'
                                ,'DT_YEAR'
                                ,'QT_RETURNED'
                                ,'VL_RETURNED'
                                ]
                    )
            ]
    ]
)

# Exportando o dataframe
df_relatorio_vendas.to_csv('./dataset/relatorio_vendas_etapa_4.csv',header=True, sep=";", index=False, encoding="utf-8")